In [3]:
# Load model directly
from transformers import AutoProcessor, BlipForImageTextRetrieval

processor = AutoProcessor.from_pretrained("Salesforce/blip-itm-base-coco")
model = BlipForImageTextRetrieval.from_pretrained("Salesforce/blip-itm-base-coco")

In [11]:
from PIL import Image
import requests

# img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg'
# raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

raw_image = Image.open("../data/bus.jpg").convert("RGB")

question = "A woman and a dog sitting together in a beach."
inputs = processor(raw_image, question, return_tensors="pt")

itm_scores = model(**inputs)[0]
cosine_score = model(**inputs, use_itm_head=False)[0]

# output image caption
print("Image caption:", processor.tokenizer.decode(inputs["input_ids"][0]))


TypeError: BlipForImageTextRetrieval.forward() missing 1 required positional argument: 'input_ids'

In [12]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image

model = VisionEncoderDecoderModel.from_pretrained(
    "nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained(
    "nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained(
    "nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}


def predict_step(image_paths):
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = feature_extractor(
      images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds




config.json: 100%|██████████| 4.61k/4.61k [00:00<00:00, 9.20MB/s]
pytorch_model.bin: 100%|██████████| 982M/982M [02:05<00:00, 7.85MB/s] 
tokenizer_config.json: 100%|██████████| 241/241 [00:00<00:00, 2.15MB/s]
vocab.json: 100%|██████████| 798k/798k [00:00<00:00, 2.66MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 2.08MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 3.92MB/s]
special_tokens_map.json: 100%|██████████| 120/120 [00:00<00:00, 332kB/s]


FileNotFoundError: [Errno 2] No such file or directory: 'doctor.e16ba4e4.jpg'

In [15]:
# ['a woman in a hospital bed with a woman in a hospital bed']
predict_step(['../data/bus.jpg'])

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


['a city bus is stopped at a bus stop']